In [ ]:
#simple bert classifier

In [5]:
# !pip install transformers
# !pip install torch
# !pip install pytorch-lightning

     ------------------------------------- 776.9/776.9 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 805.2/805.2 kB 24.8 MB/s eta 0:00:00
     ------------------------------------- 365.5/365.5 kB 22.2 MB/s eta 0:00:00
     ---------------------------------------- 76.2/76.2 kB 4.1 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ---------------------------------------- 44.7/44.7 kB ? eta 0:00:00
